In [15]:
import numpy as np
from tqdm import tqdm
from scipy.stats import multivariate_normal

In [40]:
pi_theta = 1/324000.0
varcov_lambda = np.array(([0.25,0,0,0],[0,0.25,0,0],
                          [0,0,1,0], [0,0,0,1]))

In [39]:
prior_alpha = np.random.uniform(1.1, 2., size=1000)
prior_beta = np.random.uniform(-1., 1, size=1000)
prior_gamma = np.random.uniform(0., 300., size=1000)
prior_delta = np.random.uniform(-300., 300., size=1000)
prior_gen = np.vstack((prior_alpha,prior_beta,prior_gamma, prior_delta))
prior_gen = np.transpose(prior_gen)

In [38]:
#scale parameters
#epsilon_t

mille = np.linspace(start=100, stop=1000, endpoint=True, num=10)[::-1]
cent = np.linspace(start=10, stop=100, endpoint=False, num=90)[::-1]
dix = np.linspace(start=5, stop=10, endpoint=False, num=10)[::-1]
cinq = np.linspace(start=3, stop=5, endpoint=False, num=40)[::-1]
trois = np.linspace(start=0, stop=3, endpoint=False, num=300)
trois = np.delete(arr=trois, obj=0)
trois = trois[::-1]

scale_param = np.concatenate((mille, cent, dix, cinq, trois))

In [4]:
def generation(n, alpha=1.7, beta=0.9, gamma=10, delta=10):
    # Initialize samples array with zeros
    sample = np.zeros(n)
    
    # Constants that do not depend on the sample index and thus can be computed once
    if alpha != 1:
        S_alpha_beta = (1 + beta ** 2 * np.tan(np.pi * alpha / 2) ** 2) ** (1 / (2 * alpha))
        B_alpha_beta = (1 / alpha) * np.arctan(beta * np.tan(np.pi * alpha / 2))

    for i in range(n):
        U = np.random.uniform(-np.pi/2, np.pi/2)
        W = -np.log(1 - np.random.uniform(0,1))
        
        # Handle the case alpha = 1 separately
        if alpha != 1:
            part1 = np.sin(alpha * (U + B_alpha_beta)) / (np.cos(U) ** (1 / alpha))
            part2 = (np.cos(U - alpha * (U + B_alpha_beta)) / W) ** ((1 - alpha) / alpha)
            sample[i] = S_alpha_beta * part1 * part2
        else:
            sample[i] = (2 / np.pi) * ((np.pi / 2 + beta * U) * np.tan(U) - beta * np.log((np.pi / 2 * W * np.cos(U))/(np.pi+beta*U)))

    # Apply scaling and location shifting
    sample = gamma * sample + delta
    return sample

In [11]:
def gaussian_ker(u=0, y=0, epsilon=1):
    """gaussian kernel for weights
    
    Parameters
    -------------------
    y : float, or array-like
    the point we have, the output

    u : float, or array-like
    the point from which we want to calculate a weight

    epsilon : int, float
    the scale parameter for which we want to compute the kernel
    ----------
    """

    w = (1/np.sqrt(2*np.pi*(epsilon**2)))*np.exp(-(np.abs((u-y)))**2/(2*(epsilon**2)))/epsilon
    return w

In [6]:
def zolotarev_transfo(sample, xi=0.25):
    """function to use for the estimation based on the zolotarev transformation

    Parameters
    --------------------------
    Sample : array-like
    Sample to do the transformation on

    xi : int, float
    The constant used in the transformation
    --------------------------
    """
    if xi<=0 or xi>1/2 :
        raise ValueError('Xi must be between 0 and 1/2')
    taille = len(sample)
    Z = []
    for i in range(int(taille/3)):
        transfo = sample[3*i-2] - xi*sample[3*i-1] - (1 - xi)*sample[3*i]
        Z.append(transfo)
    V = []
    U = []
    for i in range(len(Z)):
        V.append(np.log(np.abs(Z[i])))
        U.append(np.sign(sample[i]))
    V = np.array(V)
    U = np.array(U)
    S_U_squared = (np.std(U))**2
    S_V_squared = (np.std(V))**2
    nu_tilde = (6/(np.pi)**2)*S_V_squared - (3/2)*S_U_squared + 1
    etha_hat = np.mean(U)
    tau_hat = np.mean(V)
    nu_hat = 0
    if nu_tilde > ((1+np.abs(etha_hat))**2)/4:
        nu_hat = nu_tilde
    else:
        nu_hat = ((1+np.abs(etha_hat))**2)/4
    delta_hat = np.mean(sample)
    S_2 = np.array((nu_hat, etha_hat, tau_hat, delta_hat))
    return S_2

In [9]:
default = generation(n=1000)

In [13]:
def pi_lf(epsilon_t, sample=default, alpha=1.7, beta=0.9, gamma=10, delta=10, possession=False):
    """function to have the likelihood free density given in the article, the data we create if there is
    no data provided is based on the parameters of 'generation', so for any comparaison between the values of theta,
    those are the parameters to change

    Parameters
    -----------------
    N : int
    number of priors we want to generate
        
    alpha, beta, gamma, delta : int, float
    the parameters with which we want to compute the true data if we don't already have it


    sample : array-like
    the data we observe

    summary_statistic : array-like
    summary statistic used to make tests on the distance between two datasets

    epsilon_t : int, float
    scale parameter, determines 

    Possession : boolean
    by default, set to false, to determine if we want to generate the data or if we already have it

    method : str
    the method with which we would compute the summary statistics
    -------------------
    """
    if possession == False :
        data = generation(n=1000)
    else :
        data = sample
    true_param = zolotarev_transfo(sample=data)
    proposal = generation(n=1000, alpha=alpha, beta=beta, gamma=gamma, delta=delta)
    zolo_proposal = zolotarev_transfo(sample=proposal)
    weight = gaussian_ker(u=np.linalg.norm(true_param-zolo_proposal), epsilon=epsilon_t)
    pi_lf = weight*pi_theta
    return pi_lf

In [36]:
weight_gen = np.zeros((1000))

In [ ]:
weight_gen[i] = pi_lf(epsilon_t=periode_t, theta_i, sample=y)/mutation_density(theta_i, t)

In [44]:
def mutation_creation(t):
    """a function to sample a theta_t^(i) as in the paper, according to the distribution of the mutation
    kernel

    Parameters
    -------------------------
    t : int,
    period at which we do this one
    --------------------------
    """
    weighted_random = np.zeros((1000,4))
    for i in range(1000):
        weighted_random[i] = weight_gen[i]*multivariate_normal.rvs(mean=resampled[t-1], cov=varcov_lambda, size=1)
    mt_theta = np.sum(a=weighted_random, axis=0)
    return mt_theta

In [45]:
def mutation_density(theta, t):
    """function that evaluates the pdf of the kernel for a given theta

    Parameters
    ----------------------
    t : int, 
    period at which we do this evaluation

    theta : 1-D array, 
    the parameter of which we will estimate the probability density
    ----------------------
    """
    weighted_estimated = np.zeros(1000)
    for i in range(1000):
        weighted_estimated[i] = weight_gen[i]*multivariate_normal.pdf(theta, mean=resampled[t-1],
                                                                      cov=varcov_lambda)
    mt_theta = np.sum(a=weighted_estimated, axis=0)
    return mt_theta
    